In [1]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import shapefile as sf
import pandas as pd

In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [3]:
ee.Initialize(project = 'sediment-473319')

In [4]:
aoi=ee.Geometry.Rectangle([31.71630970,-24.05487960,31.72086902,-24.05871824])

In [74]:
# Actual analysis
# Sensors when in the rivers in 2021, start there and run to present.
start = pd.date_range(start= '2025-01-01' , end='2025-12-27' , 
              freq='5d')
end = pd.date_range(start='2025-01-05' , end='2025-12-31' , 
              freq='5d')
#print(len(start))
#print(len(end))
dates = pd.DataFrame ({'start': start ,  'end': end})
print(dates)

        start        end
0  2025-01-01 2025-01-05
1  2025-01-06 2025-01-10
2  2025-01-11 2025-01-15
3  2025-01-16 2025-01-20
4  2025-01-21 2025-01-25
..        ...        ...
68 2025-12-07 2025-12-11
69 2025-12-12 2025-12-16
70 2025-12-17 2025-12-21
71 2025-12-22 2025-12-26
72 2025-12-27 2025-12-31

[73 rows x 2 columns]


In [63]:
# Test dates
start = pd.date_range(start= '2017-08-13' , end='2017-08-13' , 
              freq='5d')
end = pd.date_range(start='2017-08-18' , end='2017-08-18' , 
              freq='5d')
dates = pd.DataFrame ({'start': start ,  'end': end})
print(dates)

       start        end
0 2017-08-13 2017-08-18


In [75]:
# Define function
def testGEE(startDate, endDate):
    # Define source data
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .select('B1','B2','B3', 'B4', 'B5','B6','B7','B8','B9','B11','B12')\
        .filterBounds(aoi)
    mosaic = image.median().reproject(crs='EPSG:32736', scale=10) # This allows us to set the resolution.
    band_arrs = mosaic.sampleRectangle(region=aoi, defaultValue = 0)
    # Get individual band arrays.
    band_arr_b2 = band_arrs.get('B2')   # Blue
    band_arr_b3 = band_arrs.get('B3')   # Green
    band_arr_b4 = band_arrs.get('B4')   # Red
    # Transfer the arrays from server to client and cast as np array.
    b2 = np.array(band_arr_b2.getInfo())   # b2  Blue
    b3 = np.array(band_arr_b3.getInfo())   # b3  Green
    b4 = np.array(band_arr_b4.getInfo())   # b4  Red
    # Export RGB
    np_arr_b4 = np.expand_dims(b4, 2)
    np_arr_b3 = np.expand_dims(b3, 2)
    np_arr_b2 = np.expand_dims(b2, 2)
    rgb_img = np.concatenate((np_arr_b4, np_arr_b3, np_arr_b2), 2)
    rgb_img = (255*((rgb_img)/3000)).astype('uint8')
    plt.imshow(rgb_img)
    plt.show()

In [ ]:
def pulldata(startDate, endDate):
    "COPERNICUS/S2_SR_HARMONIZED"
    # Defining image collection to draw from
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .select('B1','B2','B3', 'B4', 'B5','B6','B7','B8','B9','B11','B12')\
        .filterBounds(aoi)\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
    med = image.median().reproject(crs='EPSG:32736', scale=10) # This allows us to set the resolution.
    #mosaic = med.sampleRectangle(region=aoi, defaultValue = 0)
    clipped_image = med.clip(aoi)#.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12'])

    # Exporting image
    task = ee.batch.Export.image.toDrive(image=clipped_image,
                                     description=f'River_Images',
                                     scale=15,
                                     region=aoi,
                                     fileNamePrefix=f'Balule_{startDate}',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     folder='Turbidity_Images')
    task.start()

In [81]:
def getDates(startDate, endDate):
    "COPERNICUS/S2_SR_HARMONIZED"
    # Defining image collection to draw from
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .select('B1','B2','B3', 'B4', 'B5','B6','B7','B8','B9','B11','B12')\
        .filterBounds(aoi)\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
    med = image.median().reproject(crs='EPSG:32736', scale=10) # This allows us to set the resolution.
    #mosaic = med.sampleRectangle(region=aoi, defaultValue = 0)
    clipped_image = med.clip(aoi)#.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12'])

    # Determine date of image
    acquire = image.get('system:time_start')#.getInfo()
    acquire_date = ee.Date(acquire).format('YYYY-MM-dd HH:mm:ss')
    print(f"Acquisition date: {acquire_date.getInfo()}")

In [82]:
# Run over all dates
for i in range(len(dates)):
    # testGEE(dates["start"][i], dates["end"][i])
    # pulldata(dates["start"][i], dates["end"][i])
    getDates(dates["start"][i], dates["end"][i])

EEException: Date: Parameter 'value' is required and may not be null.

In [ ]:
#Olifants River at Balule 
#7377-609302